In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-241842
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-241842


In [44]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ml-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
 
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [51]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--max_iter': choice(100, 500, 1000),
        '--C': loguniform(1e-4, 1e4)
    }
) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4) ### YOUR CODE HERE ###

In [52]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [53]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_0f389e54-1834-412b-9f43-10e895e4cd8d
Web View: https://ml.azure.com/runs/HD_0f389e54-1834-412b-9f43-10e895e4cd8d?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-241842/workspaces/quick-starts-ws-241842&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-26T15:02:41.252767][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-09-26T15:02:41.8717577Z][SCHEDULER][INFO]Scheduling job, id='HD_0f389e54-1834-412b-9f43-10e895e4cd8d_0' 
[2023-09-26T15:02:42.0043622Z][SCHEDULER][INFO]Scheduling job, id='HD_0f389e54-1834-412b-9f43-10e895e4cd8d_1' 
[2023-09-26T15:02:42.1264613Z][SCHEDULER][INFO]Scheduling job, id='HD_0f389e54-1834-412b-9f43-10e895e4cd8d_2' 
[2023-09-26T15:02:42.2300544Z][SCHEDULER][INFO]Scheduling job, id='HD_0f389e54-1834-412b-9f43-10e895e4cd8d_3' 
[2023-09-26T15:02:42.2795878Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_0f389e54-1834-412b-9f43-10e895e4cd8d_0

{'runId': 'HD_0f389e54-1834-412b-9f43-10e895e4cd8d',
 'target': 'ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-09-26T15:02:40.421206Z',
 'endTimeUtc': '2023-09-26T15:25:20.220766Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3d78c111-761e-4738-9f21-e23201ec3bc9',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '0.9059237679048313',
  'best_child_run_id': 'HD_0f389e54-1834-412b-9f43-10e895e4cd8d_11',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_0f389e54-1834-412b-9f43-10e895e4cd8d_11'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryVal

In [54]:
assert(hyperdrive_run.get_status() == "Completed")

In [55]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [56]:
best_run.get_metrics()

{'Regularization Strength:': 3122262070046.714,
 'Max iterations:': 500,
 'Accuracy': 0.9059237679048313}

In [57]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-09-26-15',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_link)

In [11]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=100,
    task="classification",
    primary_metric='accuracy',
    X=x,
    y=y,
    n_cross_validations=5)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_8fb93061-3da9-4b5b-b10e-b2bb1df0fabe,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/09/26 13:22:10 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [14]:
automl_run.wait_for_completion(show_output = True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_8fb93061-3da9-4b5b-b10e-b2bb1df0fabe,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_8fb93061-3da9-4b5b-b10e-b2bb1df0fabe',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-09-26T13:21:19.925182Z',
 'endTimeUtc': '2023-09-26T14:18:28.44693Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-241842","workspace_name":"quick-starts-ws-241842","region":"southcentralus","compute_target":"local","spark_service":null

In [15]:
assert(automl_run.get_status() == "Completed")

In [16]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_automl_model = automl_run.get_output()

In [17]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_8fb93061-3da9-4b5b-b10e-b2bb1df0fabe_41,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [39]:
fitted_automl_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('30', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))], verbose=False))

In [43]:
fitted_automl_model.steps[1][1].__dict__

{'estimators': [('30',
   Pipeline(memory=None,
            steps=[('standardscalerwrapper',
                    StandardScalerWrapper(copy=True, with_mean=False, with_std=False)),
                   ('xgboostclassifier',
                    XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))],
            verbose=False)),
  ('24',
   Pipeline(memory=None,
            steps=[('standardscalerwrapper',
                    StandardScalerWrapper(copy=True, with_mean=False, with_std=False)),
                   ('xgboostclassifier',
                    XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', 

In [21]:
joblib.dump(fitted_automl_model, 'automl.joblib')

['automl.joblib']

In [58]:
aml_compute.delete()